In [ ]:
!pip install openpyxl mlflow pandas scikit-learn==1.5.2 oracledb python-dotenv tensorflow scikeras xgboost lightgbm seaborn --proxy http://172.25.156.90:3128

In [ ]:
from db_utils import connect_to_oracle, get_data_for_anomaly_type
from pipeline_utils import create_pipelines, evaluate_pipeline, evaluate_pipeline_oodd, create_pipelines_for_probs, evaluate_pipeline_from_probs
from sklearn.model_selection import train_test_split
import sys
sys.path.append('./src/oodd_detectors')
sys.path.append('./src/preprocessors')
from target_preprocessor import create_y_pipeline
import mlflow
from continuous_OODD import OODDContinuousModel
from categorical_OODD import OODDCategoricalModel, OODDSmoothedCategoricalModel
from counter_OODD import CountBasedClassifier, FallbackCountClassifier
from plot_utils import plot_target_distribution
import pandas as pd
# needs scikit-learn version 1.5.2 (default 1.6.1 throws 'super' object has no attribute '__sklearn_tags__'.
# from scikeras.wrappers import KerasClassifier
connection, cursor = connect_to_oracle()

In [ ]:
anomaly_type = "IM_SP_REF"
df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=50000)
from plot_utils import plot_target_distribution

plot_target_distribution(df, y_col)

In [ ]:
y_pipe = create_y_pipeline(target_col=y_col, anomaly_type=anomaly_type)
# y = y_pipe.fit_transform(df[y_col].copy())
df = y_pipe.fit_transform(df)
y = df[y_col]

# df.drop(columns=[y_col], inplace=True)
X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.1, random_state=42)

for pipeline in create_pipelines(X_cols):
    # Evaluate the pipeline
    evaluate_pipeline(pipeline, X_train, y_train,
                      X_test, y_test, verbose=False)

In [ ]:
# df, X_cols, y_col = get_data_for_anomaly_type(cursor, 'HEATING_TYPE', limit=50000)

X_train, X_test = train_test_split(df, test_size=0.1, random_state=42)
print("\nOODDContinuousModel:")
evaluate_pipeline_oodd(OODDContinuousModel(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2' )
print("\nOODDCategoricalModel:")
evaluate_pipeline_oodd(OODDCategoricalModel(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2')
print("\nOODDSmoothedCategoricalModel:")
evaluate_pipeline_oodd(OODDSmoothedCategoricalModel(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2')
print("\nCountBasedClassifier:")
evaluate_pipeline_oodd(CountBasedClassifier(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2')
print("\nFallbackCountClassifier:")
evaluate_pipeline_oodd(FallbackCountClassifier(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.1, random_state=42)

for pipeline in create_pipelines_for_probs(X_cols):
    print('\n' + pipeline.steps[-1][0])
    evaluate_pipeline_from_probs(pipeline, X_train, y_train,  X_test, y_test, threshold=0.5, target_col=y_col)

In [ ]:
anomaly_type = "EX_SP_REF"
df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=100000)

plot_target_distribution(df, y_col)
y_pipe = create_y_pipeline(target_col=y_col, anomaly_type=anomaly_type)
# y = y_pipe.fit_transform(df[y_col].copy())
df = y_pipe.fit_transform(df)
y = df[y_col]
plot_target_distribution(df, y_col)

X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.1, random_state=42)

for pipeline in create_pipelines(X_cols):
    # Evaluate the pipeline
    evaluate_pipeline(pipeline, X_train, y_train,
                      X_test, y_test, verbose=False)

# df, X_cols, y_col = get_data_for_anomaly_type(cursor, 'HEATING_TYPE', limit=50000)

X_train, X_test = train_test_split(df, test_size=0.1, random_state=42)
print("\nOODDContinuousModel:")
evaluate_pipeline_oodd(OODDContinuousModel(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2' )
print("\nOODDCategoricalModel:")
evaluate_pipeline_oodd(OODDCategoricalModel(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2')
print("\nOODDSmoothedCategoricalModel:")
evaluate_pipeline_oodd(OODDSmoothedCategoricalModel(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2')
print("\nCountBasedClassifier:")
evaluate_pipeline_oodd(CountBasedClassifier(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2')
print("\nFallbackCountClassifier:")
evaluate_pipeline_oodd(FallbackCountClassifier(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2')

X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.1, random_state=42)

for pipeline in create_pipelines_for_probs(X_cols):
    print('\n' + pipeline.steps[-1][0])
    evaluate_pipeline_from_probs(pipeline, X_train, y_train,  X_test, y_test, threshold=0.5, target_col=y_col)

In [ ]:
anomaly_type = "EX_SP_REF_WITHOUT_OD"
df, X_cols, y_col = get_data_for_anomaly_type(
    cursor, anomaly_type, limit=100000)

plot_target_distribution(df, y_col)
y_pipe = create_y_pipeline(target_col=y_col, anomaly_type=anomaly_type)
# y = y_pipe.fit_transform(df[y_col].copy())
df = y_pipe.fit_transform(df)
y = df[y_col]
plot_target_distribution(df, y_col)

X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.1, random_state=42)

for pipeline in create_pipelines(X_cols):
    # Evaluate the pipeline
    evaluate_pipeline(pipeline, X_train, y_train,
                      X_test, y_test, verbose=False)

# df, X_cols, y_col = get_data_for_anomaly_type(cursor, 'HEATING_TYPE', limit=50000)

X_train, X_test = train_test_split(df, test_size=0.1, random_state=42)
print("\nOODDContinuousModel:")
evaluate_pipeline_oodd(OODDContinuousModel(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2' )
print("\nOODDCategoricalModel:")
evaluate_pipeline_oodd(OODDCategoricalModel(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2')
print("\nOODDSmoothedCategoricalModel:")
evaluate_pipeline_oodd(OODDSmoothedCategoricalModel(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2')
print("\nCountBasedClassifier:")
evaluate_pipeline_oodd(CountBasedClassifier(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2')
print("\nFallbackCountClassifier:")
evaluate_pipeline_oodd(FallbackCountClassifier(X_cols, y_col), X_train, X_test, target_col=y_col,type='categorical2')

X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.1, random_state=42)

for pipeline in create_pipelines_for_probs(X_cols):
    print('\n' + pipeline.steps[-1][0])
    evaluate_pipeline_from_probs(pipeline, X_train, y_train,  X_test, y_test, threshold=0.5, target_col=y_col)